In [1]:
# 分析用
import numpy as np
import pandas as pd

# プロット用
import matplotlib.pyplot as plt
%matplotlib inline

# 機械学習用
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit

In [2]:
df = pd.read_csv("train.csv")
df = df.loc[:, ["PassengerId", "Survived", "Pclass", "Sex", "Age", "SibSp", "Parch"]]
df = df[np.isnan(df['Age']) == False]

In [3]:
df['Sex'] = df['Sex'].map(lambda x: 0 if x == "male" else 1)
df['Pclass_1'] = df['Pclass'].map(lambda x: 1 if x == 1 else 0)
df['Pclass_2'] = df['Pclass'].map(lambda x: 1 if x == 2 else 0)
df = df.drop("Pclass", axis = 1)

In [4]:
#各変数を正規化
df2 = df.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
df2["PassengerId"] = df['PassengerId']
df2['Survived'] = df['Survived']

In [5]:
df2_except = df2.drop("Survived", axis=1).drop("PassengerId", axis=1)
X = df2_except.as_matrix()
Y = df2['Survived'].as_matrix()

In [12]:
clf = LogisticRegression()
clf.fit(X, Y)
df_result = pd.DataFrame({"Name":df2_except.columns,"Coefficients":clf.coef_[0]}).sort_values(by='Coefficients')
df_result.ix[:,['Name', 'Coefficients']]

,Name,Coefficients
1,Age,-2.301550
2,SibSp,-1.129613
3,Parch,-0.184526
5,Pclass_2,1.045075
4,Pclass_1,2.211635
0,Sex,2.450932


In [7]:
clf.intercept_[0]

-0.49047649204278876

In [8]:
#予測してみる
df2['eq'] = -0.490 - 2.3016*df2['Age'] + 2.451 * df2['Sex'] + 2.212 * df2['Pclass_1'] + 1.045 * df2['Pclass_2'] - 1.13 * df2['SibSp'] - 0.1845 * df2['Parch']

In [9]:
df2['e^eq'] = np.exp(df2['eq'])
df2['Survived-predicted'] = df2['e^eq'] / (1 + df2['e^eq'])
df_pre = df2.ix[:,['Survived', 'Survived-predicted']]
df_pre['prediction'] = df_pre['Survived-predicted'].map(lambda x : 1 if x > 0.5 else 0)

In [10]:
r = {}
r["1-1"] = np.logical_and(df_pre['Survived'] == 1, df_pre['prediction'] == 1).value_counts(sort=False)[1]
r["1-0"] = np.logical_and(df_pre['Survived'] == 1, df_pre['prediction'] == 0).value_counts(sort=False)[1]
r["0-1"] = np.logical_and(df_pre['Survived'] == 0, df_pre['prediction'] == 1).value_counts(sort=False)[1]
r["0-0"] = np.logical_and(df_pre['Survived'] == 0, df_pre['prediction'] == 0).value_counts(sort=False)[1]
r

{'0-0': 362, '0-1': 62, '1-0': 85, '1-1': 205}

In [11]:
#正解率
(r['0-0'] + r['1-1']) / (r['0-0'] + r['1-1'] + r["1-0"] + r["0-1"]) *100

79.411764705882348